# Author: Petros Polychronis

In [1]:
import pymongo
from pymongo import MongoClient
import json
import pprint
import os
import pandas as pd
client = MongoClient("mongodb://mongo:27017")
db = client.db
# Connect
if("db" not in client.list_database_names() or "zipcodes" not in db.list_collection_names() or "prescriptions" not in db.list_collection_names()):
    os.system('python HW2-import.py')

# Checking if the data is correctly inserted:
[db.zipcodes.count_documents({}), db.prescriptions.count_documents({})]

[29353, 239930]

# Brief collection inspection
## *zipcodes* collection schema

In [2]:
db.zipcodes.find_one()

{'_id': '01001',
 'city': 'AGAWAM',
 'loc': [-72.622739, 42.070206],
 'pop': 15338,
 'state': 'MA'}

Each document in the *zipcodes* collection seems to be about cities in the US and has the following fields:
 - *_id*: mandatory unique document identifier (string)
 - *city*: name of the city (string)
 - *loc*: 2D location coordinates of the city's geographical center (list of numercial values)
 - *pop*: city's population (numerical value)
 - *state*: abbreviation of the state that the city belongs to (string)

## *presciprtions* collection schema

In [3]:
db.prescriptions.find_one()

{'_id': ObjectId('62c813d3efa44ab6860b2f0c'),
 'cms_prescription_counts': {'DOXAZOSIN MESYLATE': 26,
  'MIDODRINE HCL': 12,
  'MEGESTROL ACETATE': 11,
  'BENAZEPRIL HCL': 11,
  'METOLAZONE': 73,
  'NOVOLOG': 12,
  'DIAZEPAM': 24,
  'HYDRALAZINE HCL': 50,
  'SENSIPAR': 94,
  'LABETALOL HCL': 28,
  'PREDNISONE': 40,
  'CALCITRIOL': 79,
  'HYDROCODONE-ACETAMINOPHEN': 64,
  'HYDROCHLOROTHIAZIDE': 59,
  'LOSARTAN-HYDROCHLOROTHIAZIDE': 14,
  'FENOFIBRATE': 14,
  'MINOXIDIL': 14,
  'MELOXICAM': 29,
  'ATENOLOL': 62,
  'CARISOPRODOL': 40,
  'GABAPENTIN': 35,
  'OMEPRAZOLE': 35,
  'KLOR-CON M10': 20,
  'LANTUS': 20,
  'AMLODIPINE BESYLATE': 175,
  'CARVEDILOL': 36,
  'LOSARTAN POTASSIUM': 41,
  'IRBESARTAN': 11,
  'NIFEDICAL XL': 32,
  'NIFEDIPINE ER': 51,
  'LEVOTHYROXINE SODIUM': 12,
  'POTASSIUM CHLORIDE': 30,
  'FUROSEMIDE': 162,
  'GLYBURIDE': 16,
  'CLONIDINE HCL': 43,
  'TEMAZEPAM': 41,
  'SPIRONOLACTONE': 50,
  'LOVASTATIN': 11,
  'LISINOPRIL': 44,
  'PANTOPRAZOLE SODIUM': 13,
  'CALCIU

Each document in the *prescription* collection seems to be about doctors and prescriptions they have given, and has the following fields:
 - *_id*: mandatory unique document identifier (ObjectId)
 - *'cms_prescription_counts'*: A field that contains an embedded document. The document contains drug names as fields (string) mathced with the prescribed amounts (numeric)
 - *provider_variables*: A field that also contains an embedded document providing information about the doctor. It contains the following fields:
     - *settlement_type* (string)
     - *gengeric_rx_count* (numeric)
     - *specialty*: The doctor's specialisation area (string)
     - *years_practicing* (numeric)
     - *gender* (string)
     - *region* (string)
     - *brand_name_rx_count (numeric)
     
  - *npi*: The doctor's unique identifier number (string)

# Queries for "Zip codes"

## Q1

Count the total number of cities in Washington state (code: "WA")


## Q1 Solution

### Solution approach
The total number of cities in the state of Washington can be calculated in two steps. First, we match all documents from the collection that have 'WA' as value in their *state* field, and second, we count how many *distinct* (different) values exist for the *city* field among those (filtered) documents.

This can be done in various ways. The standard approach would be to create a *pipeline*, a list of sequential manipulations/queries to be executed on the *zipcodes* collection, and apply it via the `aggregate()` method. We will see this technique multiple times later on. For now, we can get to the desired results in a (syntactically) simpler way by using pymongo's built-in functions `find()` and `disticnt()`.

Steps:
1. Apply the `find({condition})` method on the *zipcodes* collection to match all documents pertaining to the state of Washington. This returns a cursor object (cursor: object used to iterate over the results of the executed queries).

2. Apply the `distinct("field of interest")` function to return the  different values found for the *city* field within those documents. This mehtod is applied subsequently on the results of the `.find()` query and returns a list object.

3. We can get the length of the list  by using Python's built-in function `len()`.

In [4]:
# find the length of the list containing all distinct city names in WA.
len(db.zipcodes.find({'state':'WA'}).distinct("city"))

397

## Q1 Answer
There are in total 397 different values for the *city* field in the documents having 'WA' as *state*.

### Cross-check
Is the number of different *cities* equal to the number of documents having 'WA' as *state*? Intuitively,  we would expect the two numbers to be equal, since a) it is hard to imagine a state having different cities with the same name and b) each entry gives information about a city, and in a well-maintained DB we would not want to have duplicate entries.

If we cacluate the number of documents associated with the state of Washington, we see that there are 484 documents, but only 397 different cities. Further inspection could reveal whether there are duplicate entries or not, which we would ideally want to filter out for future calculations, such as population sums.

In [5]:
# find documents with state = 'WA'
db.zipcodes.count_documents({'state':'WA'})

484

By inspecting a handful of documents, it becomes clear that the information is not duplicated. For example, while there are multiple documents for the *city* "BELLEVUE", each refers to a slightly different location and the populations are rather different. Therefore, we assume that these entries are complentary for the city of "BELLEVUE" (and for the rest of the cities where the same occurs) and we would want to factor them in in future calculations.

In [6]:
q1_list = list(db.zipcodes.find({'state':'WA'}))

for i in range(8):
    print("city", q1_list[i]['city'], "loc", q1_list[i]['loc'], "population", q1_list[i]['pop'] )

city ALGONA loc [-122.270057, 47.316339] population 22846
city AUBURN loc [-122.206741, 47.30503] population 38163
city FEDERAL WAY loc [-122.311726, 47.3203] population 34573
city BEAUX ARTS loc [-122.207371, 47.619899] population 23724
city BELLEVUE loc [-122.166288, 47.614961] population 14297
city BELLEVUE loc [-122.155179, 47.561425] population 26775
city BELLEVUE loc [-122.142572, 47.617443] population 21887
city BELLEVUE loc [-122.116173, 47.611468] population 24046


## Q2

Find the total population of each state (i.e., sort states by their population in the ascending
order).

## Solution approach

We can find the population of each state using a "groupby"-like operation using the MongoDB's `$group` operator. Specifically, we will group by the *state* field  as key and sum the populations of all documents with the same *state* value. Finally, we can sort the results using the `$sort` operator.

Before implementing our solution, we perform a few sanity checks on the field values needed for the calculations.

## Sanity Checks 


### Check 1
Verfiy that all population entries are numbers (not strings etc.)

In [7]:
len(list(db.zipcodes.find({"pop": {"$type": "int"}})))

29353

This is a convenient way to ascertain that all documents have integers as values in their *pop* field (if not all values were integers, we would have to search deeper as there are other numeric data types as well). The number of documents contained in the *zipcodes* collection is indeed 29353 (we checked that at the start). Another way to double-check this is with Python's `assert` which verifies that a logical expression is *True*.

In [8]:
assert len(list(db.zipcodes.find({"pop": {"$type": "int"}}))) == db.zipcodes.count_documents({})

### Check 2
Verfiy that all population entries are >0

In [9]:
len(list(db.zipcodes.find({"pop":{"$lt": 0}})))

0

A list lengh of 0 signals that no document satisfying the condition has been found.

We can implement  our approach with a `pipeline`. Created as a list of actions/queries in the format `pipeline = [{query 1}, {query 2},...,{query n}]`, the pipeline is passed as argument in the `.aggreate()` method to get executed. The returned object is a cursor.

Steps:  
1. Pipeline
    - Group by *state* and sum populations: `{"$group": {"_id": "$state", "population": {"$sum": '$pop'}}}`. The group by key is always given as *"_id"* and will be the values ($) of the field *state*. The population of each state will be returned as new field named "population"and will be calculated as the sum of all populations (field *pop* )belonging to the same *state* from the original *zipcodes* collection.
    
    - Sort results in ascending order (by population)) using the query syntax: `{"$sort": {"population": 1, "_id": 1}}` We rely on the `$sort` operator and pass *population* as the field that we want to order by in the format: `{"field name" : 1}`. 1 is used for ascending order and -1 for descending order. We can also order by multiple fields, which is used as a tie-breaker. In the event of population ties, we will sort the cities by `"_id": 1`, i.e., in ascending lexicographic order.
    
2. Applying the *pipeline* via the `.aggregate()` method returns a cursor object. We convert the latter into a list via Python's `list()` function. That creates a list of dictionaries, where each contains two key-value pairs of the form: `{'_id': state, 'population': value of state's population}`


In [10]:
pipeline = [
    {"$group": {"_id": "$state", "population": {"$sum": '$pop'}}},
    {"$sort": {"population": 1, "_id": 1}}
]

In [11]:
# apply pipeline
states_by_population = list(db.zipcodes.aggregate(pipeline))
# preview of one list element
states_by_population[0]

{'_id': 'WY', 'population': 453528}

## Q2 Answer

In [12]:
# print nicely
for i in states_by_population:
    print("State:", i['_id'], "Population:", i['population'])

State: WY Population: 453528
State: AK Population: 544698
State: VT Population: 562758
State: DC Population: 606900
State: ND Population: 638272
State: DE Population: 666168
State: SD Population: 695397
State: MT Population: 798948
State: RI Population: 1003218
State: ID Population: 1006749
State: HI Population: 1108229
State: NH Population: 1109252
State: NV Population: 1201833
State: ME Population: 1226648
State: NM Population: 1515069
State: NE Population: 1578139
State: UT Population: 1722850
State: WV Population: 1793146
State: AR Population: 2350725
State: KS Population: 2475285
State: MS Population: 2573216
State: IA Population: 2776420
State: OR Population: 2842321
State: OK Population: 3145585
State: CT Population: 3287116
State: CO Population: 3293755
State: SC Population: 3486703
State: AZ Population: 3665228
State: KY Population: 3675484
State: AL Population: 4040587
State: LA Population: 4217595
State: MN Population: 4372982
State: MD Population: 4781379
State: WA Populati

## Q3

Find the 10 closest cities to WEST BROOKLYN, IL. You might want to use the "$near" operator

## Solution Approach

The challenge of finding the 10 closest cities to WEST BROOKLYN in Illinois can be tackled with a three-step process. The first step is to find and extract the location of WEST BROOKLYN which is stored in the field *loc*. The location being now available, we can trace nearby cities using MongoDB's `$near` operator which orders all documents in the collection in ascending distance from the given startpoint (here WEST BROOKLYN's location). To be able to use the `$near`  operator, we will have to create a geospatial index, *GEO2D* based on *loc*'s field values. Finally, we limit the returned results to the 10 first (closest) documents, excluding WEST BROOKLYN itself.

## Sanity Checks

We need to make sure there is only one WEST BROOKLYN, IL entry. As shown in Q1, there can be multiple entries under the same city name. If more than one are provided, we may have to take an average value of all locations. Here, it turns out that there is only one city named WEST BROOKLYN in Illinois, so we do not have to take further action.

In [13]:
len(list(db.zipcodes.find({"city" : "WEST BROOKLYN", "state": "IL"})))

1

## Solution
Steps:

1. Extract the *loc* value from  WEST BROOKLYN, IL. Applying `find_one()`, returns a dictionary that resembles the document matching the criteria `{"city" : "WEST BROOKLYN", "state": "IL"}`. We extarct and save the coordinates in a list named *west_brooklyn_loc* using Python's dictionary syntax, i.e., dictionary['key name'] (here: ['loc'])

In [14]:
# get WEST BROOKLYN coordinates in a list
west_brooklyn_loc = db.zipcodes.find_one({"city" : "WEST BROOKLYN", "state": "IL"})['loc']

west_brooklyn_loc

[-89.190917, 41.729156]

2. import GEO2D geospatial index from pymongo.

In [15]:
from pymongo import GEO2D

3.  Create new geospatial index of type GEO2D based on the values of the field *loc* using the following syntax.

In [16]:
# crete new geospatial index of type GEO2D
db.zipcodes.create_index([("loc", GEO2D)])

'loc_2d'

4. Use `find()` with two conditions (seperated by comma) as criteria:
    - Use `$near` operator to sort documents in ascending distance from the starting coordinates *west_brooklyn_loc* : `{"loc": {"$near": west_brooklyn_loc}`
    - Exclude cities with the name "WEST BROOKLYN": `"city": {"$ne": "WEST BROOKLYN}`
    - Limit the returned results to the first closest cities. Applying the `.limit()` method, automatically converts the results from a cursor object into  a list of dictionaries.

In [17]:
#apply query
closest_places = db.zipcodes.find({"loc": {"$near": west_brooklyn_loc}, 
                  "city": {"$ne": "WEST BROOKLYN"} })\
                   .limit(10)

# preview one entry 
closest_places[0]

{'_id': '61367',
 'city': 'SUBLETTE',
 'loc': [-89.235409, 41.633144],
 'pop': 899,
 'state': 'IL'}

## Q3 Answer

In [18]:
# print nicely
for i in closest_places:
    print("State:", i['state'] + ",", "City: ", i['city']+ ",", "Location:", i['loc'])

State: IL, City:  SUBLETTE, Location: [-89.235409, 41.633144]
State: IL, City:  COMPTON, Location: [-89.087708, 41.684976]
State: IL, City:  ASHTON, Location: [-89.2086, 41.864327]
State: IL, City:  AMBOY, Location: [-89.34716, 41.704181]
State: IL, City:  FRANKLIN GROVE, Location: [-89.317112, 41.857968]
State: IL, City:  MENDOTA, Location: [-89.10828, 41.544308]
State: IL, City:  STEWARD, Location: [-89.015086, 41.847545]
State: IL, City:  LA MOILLE, Location: [-89.297024, 41.537557]
State: IL, City:  LEE, Location: [-88.971386, 41.786418]
State: IL, City:  PAW PAW, Location: [-88.967377, 41.685228]


Printing the results, we verify that:
1. The 10 closest cities are also in Illinois (sanity check)
2. There are no duplicate entries for the 10 cities listed (we may have had to aggregate them into 1 entry)

## Q4

Considering the `region` of each US state, according to this source, find the total population in
each of the four regions (West, South, Midwest, and Northeast)

## Solution approach

A region field missing from the *zipcodes* collection, we will resort to creating ourselves a new field based on the *state* values and the mapping of states to regions as indicated [here](https://en.wikipedia.org/wiki/List_of_regions_of_the_United_States). Once we have created a new field that pinpoints the region of each document in the collection, we can group by the  newly created region and sum the *pop* values.

## Sanity Check
Our approach relies on using  *state* values to assign each document to a region. A potential issue arises if not all documents have a *state* field. We can count the number of documents that have a *state* attribute and compare that to the total number of documents using the following syntax.

In [19]:
assert len(list(db.zipcodes.find({"state":{"$exists":True}}))) == db.zipcodes.count_documents({})

It turns out that all documents have a *state* field.

Steps:

1. We create a list that contains all states per region. To be precise, we can create n-1 lists (for n=4 regions) and define a default region which will be assigned to all documents not belonging to the n-1 regions.

In [20]:
# crete lists for each region with the states belonging to them
west = ['AZ', 'NM', 'WA', 'OR', 'CA', 'NV', 'UT','CO', 'WY', 'MT', 'ID', 'AK', 'HI']
midwest = ['ND', 'SD', 'NE','KS', 'MN','IA','MO', 'IL','IN','OH','MI', 'WI']
northeast = ['PA', 'NY', 'NJ', 'CT', 'RI', 'NH', 'MA', 'ME', 'VT']

2. The pipeline consists of two main actions.
    1. Assign regions to each document: We use the `$set` operator to define a new field named *region*. We make use of the `$switch` operator to create three cases (for the three lists of regions) and one default case, defined by the *default* keyword. To define a case, we use the syntax: `{"case": if-condition, "then": action (value to be assigned to field region)}` The condition checks whether the value of the *state* attribute for each document is contained with the lists west, midwest, northeast. If the condition is met, the corresponding region value is assigned to the  *region* field. If no condition is met, the value *South* is assigned.
        
    2. Group by regions and sum the populations. We group by the newly created field *region* and sum all populations: `{"$group": {"_id" : "$region", "population": {"$sum": '$pop'}}}`.
   

In [21]:
q4_pipeline = [{"$set": {"region":
             {"$switch": 
              {"branches":[
                  {"case": {"$in": ["$state", west]}, "then": "West"},
                  {"case": {"$in": ["$state", midwest]}, "then": "Midwest"}, 
                  {"case": {"$in": ["$state", northeast]}, "then": "Northeast"}], 
                  "default": "South"}}}},
                  
                  {"$group": {"_id" : "$region", "population": {"$sum": '$pop'}}}]

3. Apply th pipeline via `.aggregate()` and convert results into a list (using `list()`).

In [22]:
#apply pipeline and convert to list
population_by_region = list(db.zipcodes.aggregate(q4_pipeline))

# preview one result
population_by_region[0]

{'_id': 'Midwest', 'population': 59652438}

## Q4 Answer

In [23]:
# print nicely
for i in population_by_region:
    print("US Region:", i['_id'] + ", Population:", i['population'])

US Region: Midwest, Population: 59652438
US Region: Northeast, Population: 50807650
US Region: West, Population: 52774790
US Region: South, Population: 85173522


In [24]:
# delete new field
db.zipcodes.update_many({}, {"$unset": {"region": ""}})

## Q5

Find the 3 most populated cities for each state (in one query)

## Solution approach
We create and apply the following pipeline:

1. Pipeline. It contains the following 5 steps

    1. We group by *state* and *city* and sum the populations of each city (remember there might be multiple entries for each city): `{"$group": {"_id": {"state": "$state", "city": "$city"}, "Population": {"$sum": "$pop"}}}`
    2. We sort a) the states alphabetically, and b) the cities (for each state) by population in descending order: `{"$sort" : {"_id.state" : 1, "Population" : -1}}`
    3. We group again, this time only by *state* and aggregate the results by creating, for each state, a list (under a new field called *cities-pop*) and appending to it a dictionary for each of its cities containing the name of the city  as *city* and its population as *pop*. In other words, after step three we have for each state a list of dictionaries (for all its cities) in the format: `{'_id': 'state', 'cities-pop': [{'city':ANCHORAGE, 'pop': 183987}, {city 2},..., {last city of 'state'}]}`. The "appending" to a list action is undertaken through the `$push` operator and the insertion to the list happens according to the ordering of step 2. Particularly, the first values of the list correspond to the most populated cities.
    4. We use the `$slice` operator on the *cities-pop* field to keep only the first three entries, which correspond to the 3 most populated cities for each state: `{"$project": {"top-3": {'$slice':["$cities-pop",3]}}`. Each document, now takes the form: `{'_id': 'AK', 'top-3': [{'city': 'ANCHORAGE', 'pop': 183987}, {'city': 'FAIRBANKS', 'pop': 31379}, {'city': 'JUNEAU', 'pop': 24947}]}`
    5. Finally, we sort the results in alphabetical order according to the name of the state (this step is for formatting reasons only).

In [25]:
q5_pipeline = [
            {"$group": {"_id": {"state": "$state", "city": "$city"}, "Population": {"$sum": "$pop"}}},
            {"$sort" : {"_id.state" : 1, "Population" : -1}},
            {"$group": {'_id': "$_id.state", "cities-pop": {"$push": {"city":"$_id.city", "pop": "$Population"}}}},
            {"$project": {"top-3": {'$slice':["$cities-pop",3]}}},
            {"$sort" : {"_id":1}}
]

2. Apply the pipeline and preview the results

In [26]:
# apply pipeline
q5 = list(db.zipcodes.aggregate(q5_pipeline))
# preview results
q5[0:2]

[{'_id': 'AK',
  'top-3': [{'city': 'ANCHORAGE', 'pop': 183987},
   {'city': 'FAIRBANKS', 'pop': 31379},
   {'city': 'JUNEAU', 'pop': 24947}]},
 {'_id': 'AL',
  'top-3': [{'city': 'BIRMINGHAM', 'pop': 242606},
   {'city': 'MOBILE', 'pop': 235452},
   {'city': 'MONTGOMERY', 'pop': 193811}]}]

**Note**: WASHINGTON DC is a state with two cities only, WASHINGTON and PENTAGON. The `$slice` operator is robust and returns only the first two results without raising an error. To avoid an error when printing, we have included a *try* *except* clause which circumvents this issue for WHASHINGTON DC as well.

## Q5 Answer

In [27]:
# print nicely
for i in q5:
    for j in range(3):
        try:
            print("State:", i['_id'] + "," , 
                  "City:", i['top-3'][j]['city'] + "," ,
                  "Population:", i['top-3'][j]['pop'])
        except:
            None
            
    print("\n")

State: AK, City: ANCHORAGE, Population: 183987
State: AK, City: FAIRBANKS, Population: 31379
State: AK, City: JUNEAU, Population: 24947


State: AL, City: BIRMINGHAM, Population: 242606
State: AL, City: MOBILE, Population: 235452
State: AL, City: MONTGOMERY, Population: 193811


State: AR, City: LITTLE ROCK, Population: 192895
State: AR, City: FORT SMITH, Population: 76584
State: AR, City: JONESBORO, Population: 53532


State: AZ, City: PHOENIX, Population: 890853
State: AZ, City: TUCSON, Population: 586451
State: AZ, City: MESA, Population: 316356


State: CA, City: LOS ANGELES, Population: 2102295
State: CA, City: SAN DIEGO, Population: 1049298
State: CA, City: SAN JOSE, Population: 816653


State: CO, City: DENVER, Population: 451182
State: CO, City: COLORADO SPRINGS, Population: 340404
State: CO, City: AURORA, Population: 237454


State: CT, City: BRIDGEPORT, Population: 141638
State: CT, City: HARTFORD, Population: 139685
State: CT, City: WATERBURY, Population: 108681


State: DC,

## Q6

Find the specialty of all doctors who have prescribed "HALOPERIDOL"

## Solution approach
We first find the documents where doctors prescribed "HALOPERIDOL" and then we find all *distinct* specialities of doctors contained within those.

Steps:
1. Find the documents where "HALOPERIDOL" is contained in the embedded document under the field *cms_prescription_counts* field using the `$exists` operator. One peculiarity that we have not seen before, is that we can access fields within other fields using  a '.' to dive deeper into the tree structure of the document (e.g. `cms_prescription_counts.HALOPERIDOL`)

2. Find all distinct specialties using the `.distinct()` method, specifying *provider_variables.specialty* as field. The returned object is a list.

In [28]:
specialties = db.prescriptions.find({"cms_prescription_counts.HALOPERIDOL":{"$exists":True}})\
                    .distinct("provider_variables.specialty")

An alterantive approach to the first step is to use the `$gt` operator and ask for the value of the nested field `cms_prescription_counts.HALOPERIDOL` to be greater than zero. This trick has the advantage that it  first finds the documents where the field exists, thus covering our first filter, while performing simultaneously a sanity check since a negative prescribed amount does not make sense. We checked that the results are indeed the same.

In [29]:
specialties = db.prescriptions.find({"cms_prescription_counts.HALOPERIDOL":{"$gt":0}})\
                    .distinct("provider_variables.specialty")

In [30]:
# check that the the two conditions are equivalent in this setting
assert \
db.prescriptions.find({"cms_prescription_counts.HALOPERIDOL":{"$gt":0}})\
                    .distinct("provider_variables.specialty")\
==\
\
db.prescriptions.find({"cms_prescription_counts.HALOPERIDOL":{"$exists":True}})\
                    .distinct("provider_variables.specialty")

## Q6 Answer

There are in total 86 different specialties, as shown in the list below:

In [31]:
len(specialties)

86

In [32]:
print(specialties)

['Acute Care', 'Addiction (Substance Use Disorder)', 'Addiction Medicine', 'Addiction Psychiatry', 'Adolescent Medicine', 'Adolescent and Children Mental Health', 'Adult Health', 'Adult Medicine', 'Adult Mental Health', 'Allergy & Immunology', 'Anatomic Pathology', 'Anatomic Pathology & Clinical Pathology', 'Behavioral Neurology & Neuropsychiatry', 'Cardiovascular Disease', 'Child & Adolescent Psychiatry', 'Clinical', 'Clinical Cardiac Electrophysiology', 'Clinical Child & Adolescent', 'Clinical Neurophysiology', 'Cognitive & Behavioral', 'Community Health', 'Critical Care Medicine', 'Developmental \x96 Behavioral Pediatrics', 'Diagnostic Neuroimaging', 'Diagnostic Radiology', 'Emergency Medical Services', 'Endocrinology, Diabetes & Metabolism', 'Endodontics', 'Family', 'Family Health', 'Foot & Ankle Surgery', 'Forensic Psychiatry', 'Gastroenterology', 'General Practice', 'Geriatric Medicine', 'Geriatric Psychiatry', 'Gerontology', 'Gynecologic Oncology', 'Gynecology', 'Hematology', 'H

## Q7

Find the total number of doctors, separately for each region (in one query)

## Solution approach
We will count the number of doctors operating in each region by grouping by the *region* field, located within the *provider_variables* field and aggregate in two steps. First, we append  the *npi*s of doctors operating in the same region in a set (every value is contained only once, there are no duplicate values as per the set's definition), and then we calculate the size of the set. The size of the set gives us the total number of doctors in each region.

Steps:
1. Pipeline which consists of two steps:
    - Group by *provider_varibales.region* and aggregate by creating a set (via `$addToSet` operator) and appending there all relevant npis : `{"$group":{"_id": "$provider_variables.region", "uniqueNPIs": {"$addToSet": "$npi"}}}`
    - Project only the size of the set, which can be done via the `$size` operator: `{"$project":{"unique_doctors_per_region":{"$size":"$uniqueNPIs"}}}`
    
2. Apply pipeline

In [33]:
q7_pipeline = [
    
    {"$group":{"_id": "$provider_variables.region", "uniqueNPIs": {"$addToSet": "$npi"}}},
    {"$project":{"unique_doctors_per_region":{"$size":"$uniqueNPIs"}}}
]

In [34]:
# apply pipeline
q7 = list(db.prescriptions.aggregate(q7_pipeline))
# preview results
q7[0]

{'_id': 'Midwest', 'unique_doctors_per_region': 50077}

## Q7 Answer


In [35]:
# print nicely!
for i in q7:
    print("Region:", i['_id']+ ",", "Total Doctors:", i['unique_doctors_per_region'])

Region: Midwest, Total Doctors: 50077
Region: Northeast, Total Doctors: 59012
Region: West, Total Doctors: 50279
Region: South, Total Doctors: 80562


## Q8

Find the total amount of prescribed "ATORVASTATIN CALCIUM"

We create a pipeline which consists of two actions:
1. We filter out all  documents where ATORVASTATIN CALCIUM was not prescribed using the `$match` operator: `{"$match":{"cms_prescription_counts.ATORVASTATIN CALCIUM":{"$gt":0}}}`
2. We group by a null value (we do not need a key) and sum up the prescribed amount of all documents using the `$group` operator with the `$sum` aggregator function. : `{"$group": {"_id": "$null", "total_amount_prescribed": {"$sum": "$cms_prescription_counts.ATORVASTATIN CALCIUM"}}}`


**Note**: Even though we do not require a key in the `$group` operator, we do require the `$group` process overall. Applying the `$sum` operator with a simple `$project` operation would not give correct results, as `$sum` is designed to add elements within an array-type object (e.g., a set or a list). Here, each document has only one prescription amount, which means addition is not taking place.

In [36]:
q8_pipeline = [
    {"$match":{"cms_prescription_counts.ATORVASTATIN CALCIUM":{"$gt":0}}},
    
    {"$group": {"_id": "$null",
                          "total_amount_prescribed": {"$sum": "$cms_prescription_counts.ATORVASTATIN CALCIUM"}}}
    
   ]

In [37]:
# apply pipeline
q8 = list(db.prescriptions.aggregate(q8_pipeline))
# preview results
q8[0]

{'_id': None, 'total_amount_prescribed': 4534888}

## Q8 Answer

In [38]:
# print nicely
print("Total amount of prescribed ATORVASTATIN CALCIUM :", q8[0]['total_amount_prescribed'])

Total amount of prescribed ATORVASTATIN CALCIUM : 4534888


## Q9

Find the drug that is prescribed by the most of doctors working in "non-urban" areas. (in terms
of number of doctors who prescribed it, not the total amount of prescriptions).

## Solution approach

We create a pipeline executing the following actions:

Steps:  
1. First, we find only documents where the provider's (doctor's) *settlement_type* field is "non-urban", using the `$match` operator and the operator `$eq` to compare strings.
2. We convert the *'cm_prescription_counts'* dictionary into a list of dictionaries, each containing two entries. The name of the drug (previously a key in the dictionary) is converted into the value of a new key 'k'. Similarly, the prescribed amount is now a value to a new key called 'v'. As a result, the field *cm_prescription_counts* ends up being a list of the form: `[{'k': 'DOXAZOSIN MESYLATE', 'v': 26},...]`), for each document. This is achieved through the use of the `$objectToArray` operator. Using the `$project` operator, we discard all other fields and work only with *cm_prescription_counts*.
3. We use the `$unwind` operator to break down the aforementioned list into new, separate documents. Each new document will be composed of the '_id' attribute and only one of the prescribed drugs. A document containing two prescriptions as a list (see step 2), e.g., `[{'k': 'DOXAZOSIN MESYLATE', 'v': 26}, {'k': 'MEGESTROL ACETATE', 'v': 11}]` would now be decomposed into two new documents, the first having  only`[{'k': 'DOXAZOSIN MESYLATE', 'v': 26}]` as *cm_prescription_counts* and the second only `[{'k': 'MEGESTROL ACETATE', 'v': 11}]`.
4.  Using the  operator `$group`, we group by the name of each drug (*cms_prescription_counts.k*), and aggreate by summing 1s for each occurance. That way we count the total amount of times that the drug has been prescribed.
5. We sort the results in descending order based on the prescribed amount with the help of the `$sort` operator.
6. We limit the results to be the most prescribed drug only using the `$limit` operator.

In [39]:
q9_pipeline = [
    
    {
        "$match":{
            "provider_variables.settlement_type":{
                "$eq": "non-urban"}
        } 
    },
    
    {
        
    "$project": {
      "cms_prescription_counts": {
        "$objectToArray": "$cms_prescription_counts"
      }
    }
  },
 {
    "$unwind": "$cms_prescription_counts"
  },

{
    "$group": {
      "_id": "$cms_prescription_counts.k",
      "count": {
        "$sum": 1
      },
    }
},

{
    "$sort": {"count": -1, "_id": 1 }
},

{
    "$limit" : 1
}

]

In [40]:
# execute pipeline
q9= list(db.prescriptions.aggregate(q9_pipeline))
# preview results
q9

[{'_id': 'HYDROCODONE-ACETAMINOPHEN', 'count': 42784}]

## Q9 answer

In [41]:
# print nicely
print("Most prescribed drug in non-urban areas:", q9[0]['_id'] + ",",
      "Total prescriptions:", q9[0]['count'] )

Most prescribed drug in non-urban areas: HYDROCODONE-ACETAMINOPHEN, Total prescriptions: 42784


## Q10

Considering the region of US states (Query #4) and the region of each doctor, find the average
number of doctors per capita in each of the four regions in US

## Solution approach

There are various alternatives as to how to approach this question. First, we catch sight of the fact that the total population by region is not available in the *prescriptions* collection and that we will need to incorporate the results from Query 4 into our analysis. Second, the task at hand also shares similarities wit Query 7 in terms of finding the total number of doctors per region. We decide to take the following approach:

 - We will create a new collection in the database called *region_populations* which will contain four documents, one for each region. The format of each document will be: `{'_id': 'Region' (= e.g. 'South'),  'population' : numerical value}`. In fact, we make use of the already extracted dictionary *population_by_region* obtained in Q4, which contains that exact information in the descired format. We thus first create a new collection using `.create_colection("region_populations")` and then we proceed with populating that collection with the four documents entailed in the dictionary *population_by_region* using the method `.insert_many(population_by_region)`


In [42]:
# dictionary from Q4
population_by_region

[{'_id': 'Midwest', 'population': 59652438},
 {'_id': 'Northeast', 'population': 50807650},
 {'_id': 'West', 'population': 52774790},
 {'_id': 'South', 'population': 85173522}]

In [43]:
# create new collection
db.create_collection("region_populations")
# populate new collection
db.region_populations.insert_many(population_by_region)
# preview new collection
list(db.region_populations.find())

[{'_id': 'Midwest', 'population': 59652438},
 {'_id': 'Northeast', 'population': 50807650},
 {'_id': 'West', 'population': 52774790},
 {'_id': 'South', 'population': 85173522}]

- The second step involes creating the pipeline. It consists of 4 steps in total (5 different actions). 

    1. The first step in the pipeline is comprised of 2 actions and is identical to the one from Q7. In other words, it calculates the total number of doctors in each region. We can think the output of this first step as a list containing 4 documents that will be passed on to the next stage of the pipeline. Each one will have two fields, an *_id* which will be the name of the region and a field named *unique_doctors_per_region* containing the number of doctors. The document collection will take the form: `[{_id': 'South', 'unique_doctors_per_region: 80562}, {2nd document} {3rd}, {4th}]`
    
    2. The second step is a `$lookup` operation that appends relevant information to each of the four documents from step 1. The appended information comes from the new collection (*`'from'` region_populations*) we created previously. Specifically, we use the value of the '_id' field as the key value to match which information from the new collection (`foreignField`) should be appended to which document of the current collection (`localField`). In other words, the document pertaining to the region 'South' will get a new field (named *region_pop* via the *`'as'`* arugment) containing  information about the total population in the South from the *region_populations* collection. The collection of documents now takes the form:  `[{'_id': 'South', 'unique_doctors_per_region': 80562 , 'region_pop': [{'_id': 'South', 'population': 85173522}]}, {2nd document}, {3rd}, {4th}]`
    
    3. The third step is an adjustment we need to make to be able to easily access the value of a region's population. Currently the value of the new field `region_pop` is a an array(list)-type object, which means we cannot use the '.' notation to reach the population value as in `region_pop.population`. We can work around that using the `$unwind` operator which turns the list/array into a dictionary of values. Now we can use  `region_pop.population`.
    
    4. The last step involves projecting the division (unique_doctors_per_region / region_population). The division is achieved through the `$divide` operator as seen in the pipeline. What is more, we use the `$project` operator to get rid of the now redundant fields such as *unique_doctors_per_region* and *population*. That results into the division result being the only thing that remains on each document (along with the '_id' value which is always there by default unless explicitly specified otherwise)

In [44]:
q10_pipeline = [ 
    
{"$group":{"_id": "$provider_variables.region", "uniqueNPIs": {"$addToSet": "$npi"}}},
{"$project":{"unique_doctors_per_region":{"$size":"$uniqueNPIs"}}},
    
    {"$lookup": {
    "from": "region_populations",
    "localField": "_id",
    "foreignField": "_id",
    "as": "region_pop"}},

{'$unwind': { 'path': '$region_pop'} },
    
{"$project":{"Doctors_per_capita":{"$divide":["$unique_doctors_per_region","$region_pop.population"]}}}
]

In [45]:
# apply pipeline
q10 = list(db.prescriptions.aggregate(q10_pipeline))
# preview results
q10[1]

{'_id': 'Midwest', 'Doctors_per_capita': 0.0008394795196803188}

## Q10 answer

In [46]:
# print nicely!
for i in q10:
    print("Region:", i['_id'] + ',', "Doctors per capita:", i['Doctors_per_capita'])

Region: Northeast, Doctors per capita: 0.0011614786355991667
Region: Midwest, Doctors per capita: 0.0008394795196803188
Region: South, Doctors per capita: 0.0009458573287599872
Region: West, Doctors per capita: 0.0009527086701813499


In [47]:
# delete new collection after use
db.region_populations.drop()